# Brain tumor segmentation

- The segmentation-models-pytorch is a deep learning package used for image segmentation tasks 

In [1]:
# installing packages
# this is a very large package take your time 
pip install --default-timeout=120 segmentation-models-pytorch

In [3]:
pip install -q opencv-python # -q for quiet

In [4]:
pip install -q albumentations

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os # provides a way to interact with the operating system
import time
from glob import glob # used for pattern matching files and directories in a file system.
from tqdm import tqdm # progress indicator for tasks that involve iteration

In [3]:
import cv2 # open source computer vision package used computer vision
import numpy as np
import pandas as pd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import albumentations as A # for image augmentation 
from scipy.ndimage.morphology import binary_dilation # used for morphological operations on binary (boolean) images
import segmentation_models_pytorch as smp
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split


/tmp/ipykernel_4377/3532261297.py:7: DeprecationWarning: Please use `binary_dilation` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  from scipy.ndimage.morphology import binary_dilation # used for morphological operations on binary (boolean) images


In [4]:
import torch
from torch.optim import Adam # Adaptive Moment Estimation is an optimization algorithm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset ,DataLoader
from torchvision import transforms as T

In [7]:
# check for CUDA compatible GPU

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data Preprocessing

In [9]:
# getting files 
files_dir = '/media/danlof/USB STICK/MRI/kaggle_3m'
file_paths = glob(f'{files_dir}/*/*[0-9].tif')

# loading files 
csv_path = '/media/danlof/USB STICK/MRI/kaggle_3m/data.csv'
mri_df = pd.read_csv(csv_path)
mri_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Patient                    110 non-null    object 
 1   RNASeqCluster              92 non-null     float64
 2   MethylationCluster         109 non-null    float64
 3   miRNACluster               110 non-null    int64  
 4   CNCluster                  108 non-null    float64
 5   RPPACluster                98 non-null     float64
 6   OncosignCluster            105 non-null    float64
 7   COCCluster                 110 non-null    int64  
 8   histological_type          109 non-null    float64
 9   neoplasm_histologic_grade  109 non-null    float64
 10  tumor_tissue_site          109 non-null    float64
 11  laterality                 109 non-null    float64
 12  tumor_location             109 non-null    float64
 13  gender                     109 non-null    float64